In [50]:
import pandas as pd
import numpy as np
import re
import json
from pathlib import Path


import matplotlib.pyplot as plt

In [53]:
ROOT = Path('/Volumes/INWT/Daten_NLP/') # encrypted folder!
DATA = ROOT / '200707_aachener_zeitung.txt'
META = ROOT / 'Dokumentation_Daten.txt'

## Reading in sample data

In [58]:
raw = pd.read_csv(DATA, encoding='utf-8', error_bad_lines=False, quotechar = '"', doublequote=False) # index_col=0

# Achtung: in row 116, 180, 215, 308, 461, 625 treten Fehler auf ???

b'Skipping line 116: expected 52 fields, saw 54\nSkipping line 180: expected 52 fields, saw 53\nSkipping line 215: expected 52 fields, saw 53\nSkipping line 308: expected 52 fields, saw 55\nSkipping line 461: expected 52 fields, saw 55\nSkipping line 625: expected 52 fields, saw 53\n'


In [59]:
import re

## Creating dict with descriptions of columns from meta file

In [55]:
# Zusätzliche Information zu den Spalten 2-11 finden sich unter
# https://ga-dev-tools.appspot.com/dimensions-metrics-explorer/?

meta_dict = {}
with open(META, 'r') as fp:
    line = fp.readline()
    cnt = 1
    while line:
        print(line.strip())
        if '- (' in line:
            line = re.sub(r'- \(\d+\): ', '', line).strip()
            #print(line)
            var_name, var_description = re.split(': | - ',line)
            meta_dict[var_name] = var_description
        line = fp.readline()

Im Folgenden werden die Inhalte der im Datenexport enthaltenen Spalten beschrieben:
- Zusätzliche Information zu den Spalten 2-11 finden sich unter https://ga-dev-tools.appspot.com/dimensions-metrics-explorer/?
- tbi: to be ignored

- (1): articleId - ID (aus CMS des Publishers)
- (2): pageviews - Anzahl Seitenaufrufe
- (3): entrances - Anzahl Einstiege
- (4): exits - Anzahl Ausstiege
- (5): bounces - Anzahl Bounces
- (6): timeOnPage - Verweil- oder Lesedauer in Sekunden (insgesamt)
- (7): conversions - Anzahl Registrierungen/Abos
- (8): avgTimeOnPage - durchschnittliche Verweil- oder Lesedauer in Sekunden
- (9): stickiness - Anteil der Leser, die im Anschluss an den Artikel auf der Publisher-Seite blieben
- (10): entranceRate - Anteil der Leser, die über den Artikel auf die Publisher-Seite kamen
- (11): bounceRate - Anteil der Leser, die Publisher-Seite direkt wieder verlassen haben
- (12): sharePageviews - tbi
- (13): n - tbi
- (14): nObs - tbi
- (15): score_pageviews - tbi
- (16): s

In [46]:
with open('meta_dict.json', 'w') as f:
    json.dump(meta_dict, f)

In [6]:
len(meta_dict)

52

In [7]:
meta_dict['pageviews']

'Anzahl Seitenaufrufe'

In [14]:
for var in meta_dict.keys():
    if meta_dict[var] != 'tbi':
        print(var,':', meta_dict[var])

articleId : ID (aus CMS des Publishers)
pageviews : Anzahl Seitenaufrufe
entrances : Anzahl Einstiege
exits : Anzahl Ausstiege
bounces : Anzahl Bounces
timeOnPage : Verweil- oder Lesedauer in Sekunden (insgesamt)
conversions : Anzahl Registrierungen/Abos
avgTimeOnPage : durchschnittliche Verweil- oder Lesedauer in Sekunden
stickiness : Anteil der Leser, die im Anschluss an den Artikel auf der Publisher-Seite blieben
entranceRate : Anteil der Leser, die über den Artikel auf die Publisher-Seite kamen
bounceRate : Anteil der Leser, die Publisher-Seite direkt wieder verlassen haben
kategorie : Kategorie in der der Publisher den Artikel veröffentlicht hat
rubrik : Unter-Kategorie
subRubrik : UNter-Unter-Kategorie
titel : Titel (Publisher)
pagePath : URL
dpaGuid : ID aus dpa DWH
categoryCode : vm = "Vermischtes"
genre : dto
geoArea : Regioninfo
keywords : Keyword-Liste durch "__,__" getrennt
links : Weiterführende Links zum Artikel, die die dpa bereitstellt
subject : Subject-Liste durch "__,

In [15]:
raw.columns

Index(['articleId', 'pageviews', 'entrances', 'exits', 'bounces', 'timeOnPage',
       'conversions', 'avgTimeOnPage', 'stickiness', 'entranceRate',
       'bounceRate', 'sharePageviews', 'n', 'nObs', 'score_pageviews',
       'score_avgTimeOnPage', 'score_entranceRate', 'score_stickiness',
       'SCORE', 'kategorie', 'rubrik', 'subRubrik', 'titel', 'pagePath',
       'dpaGuid', 'categoryCode', 'creationDate', 'flextags', 'genre',
       'geoArea', 'importance', 'keywords', 'lifetime', 'limbic', 'links',
       'picture', 'service', 'subject', 'subjectBroader', 'subjectCode',
       'teaser', 'text', 'titelH1', 'titelH3', 'urgency', 'wikidata',
       'wordcount', 'category', 'ratgeber', 'format', 'dateLoaded',
       'nLoadAttempts'],
      dtype='object')

In [60]:
raw.shape

(891, 52)

In [61]:
# ignore colums with 'tbi' in meta_dict
use_columns = [var for var in meta_dict.keys() if meta_dict[var] != 'tbi']

In [62]:
len(use_columns)

29

In [63]:
df = raw[use_columns]

In [64]:
df.set_index('articleId', inplace=True)

In [65]:
df.shape

(891, 28)

In [66]:
df.head()

,pageviews,entrances,exits,bounces,timeOnPage,conversions,avgTimeOnPage,stickiness,entranceRate,bounceRate,...,geoArea,keywords,links,subject,teaser,text,titelH1,titelH3,wordcount,category
articleId,,,,,,,,,,,,,,,,,,,,,
48620281,21,7,12,7,1012,NaN,112.444444,42.857143,33.333333,33.333333,...,Deutschland,"Ratgeber__,__Joggen__,__Laufschuhe__,__Messen_...",NaN,"Gesundheit__,__Sport__,__Medizin__,__Freizeit","Wer ein bisschen durch den Park rennen möchte,...",München/Stuttgart (dpa/tmn) - Frische Luft und...,Ohne Treter läuft nichts: Die richtigen Schuhe...,NaN,769,vm
48620381,19,6,11,5,1484,NaN,185.500000,42.105263,31.578947,26.315789,...,Deutschland,"Ratgeber__,__Psychologie__,__Depressionen__,__...",Info-Telefon Depression,"Gesundheit__,__Medizin","Ewig grauer Himmel, Nässe, Kälte. Je länger de...",Berlin/Frankfurt/Main (dpa/tmn) - Der Wecker k...,Ohne Antrieb: Wann ist der Winter-Blues eine D...,NaN,441,vm
48622639,2,2,2,2,0,NaN,0.000000,0.000000,100.000000,100.000000,...,Deutschland,"Eltern__,__Kinder__,__Ratgeber__,__Baby__,__Ba...",Test-Artikel,"Familie__,__Technik",Schläfst du noch oder schreist du schon? Babyp...,Berlin (dpa/tmn) - Eltern auf der Suche nach e...,Funk statt WLAN: Klassische Babyphones weiter ...,NaN,390,vm
48623085,32,9,20,9,974,NaN,81.166667,37.500000,28.125000,28.125000,...,Deutschland,"Ratgeber__,__Stiftung Warentest__,__Spülmaschinen",Ergebnisse Spülmaschinentabs-Test Stiftung War...,"Immobilien__,__Wohnen__,__Verbraucher__,__Umwelt","Dosierfach auf, Tab rein, Klappe zu, Maschine ...",Berlin (dpa/tmn) - Spülmaschinentabs sollen kl...,Warentest: Die besten Spülmaschinen-Multitabs ...,NaN,345,vm
48623259,24,2,7,2,3797,NaN,223.352941,70.833333,8.333333,8.333333,...,Deutschland,"tmn0500__,__Lifestyle__,__Ratgeber__,__Stiftun...",NaN,"Mode__,__Kosmetik",Make-up mit toller Deckkraft und trotzdem natü...,Berlin (dpa/tmn) - Make-up hat heutzutage eine...,Ist das noch natürlich? - Stiftung Warentest p...,NaN,182,vm


In [67]:
df.describe()

,pageviews,entrances,exits,bounces,timeOnPage,conversions,avgTimeOnPage,stickiness,entranceRate,bounceRate,titelH3,wordcount
count,891.000000,891.000000,891.000000,891.000000,891.000000,0.0,891.000000,891.000000,891.000000,891.000000,0.0,891.000000
mean,35.459035,16.163861,20.362514,14.320988,2114.952862,NaN,114.667955,58.255002,28.108763,24.473906,NaN,340.406285
std,189.988507,113.578326,131.153139,102.734228,11761.128249,NaN,125.972488,24.728936,26.065194,24.799523,NaN,211.222006
min,1.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN,74.000000
25%,6.000000,1.000000,2.000000,0.000000,187.000000,NaN,45.600000,43.614130,6.250000,0.000000,NaN,171.000000
50%,10.000000,2.000000,3.000000,2.000000,494.000000,NaN,80.000000,60.000000,21.739130,18.181818,NaN,262.000000
75%,17.000000,5.000000,7.000000,4.000000,1177.500000,NaN,143.440476,75.000000,42.857143,39.144705,NaN,492.500000
max,3047.000000,1877.000000,2032.000000,1779.000000,217337.000000,NaN,1490.000000,100.000000,100.000000,100.000000,NaN,1374.000000


In [69]:
df.iloc[0] # one example

pageviews                                                       21
entrances                                                        7
exits                                                           12
bounces                                                          7
timeOnPage                                                    1012
conversions                                                    NaN
avgTimeOnPage                                              112.444
stickiness                                                 42.8571
entranceRate                                               33.3333
bounceRate                                                 33.3333
kategorie                                                 ratgeber
rubrik                                                  gesundheit
subRubrik                                                      NaN
titel                            die-richtigen-schuhe-fuers-joggen
pagePath         https://www.aachener-zeitung.de/ratgeber/gesu

In [70]:
# add a colum with place (e.g. München/Stuttgart)
df['city'] = [ re.split('\(dpa',text)[0].strip() for text in df.text ]

/Users/Sanna/miniconda3/envs/work/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [71]:
# add a column with minimal preprocessed text
for i in df.index:
    text = df.loc[i, 'text']
    preprocessed = text.replace('\n', ' ').strip() # delete linebreaks
    preprocessed = re.sub(' +', ' ', preprocessed) # just one space
    preprocessed = preprocessed.split('tmn) - ')[-1] # delete city in front of text
    df.loc[i, 'text_preprocessed'] = preprocessed

/Users/Sanna/miniconda3/envs/work/lib/python3.7/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Sanna/miniconda3/envs/work/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [74]:
df.loc[48620281, 'text'][:500]

'München/Stuttgart (dpa/tmn) - Frische Luft und Bewegung: Diese\nKombination reizt viele Menschen am Laufen. Damit die Muskeln und\nSehnen eine Joggingrunde gut überstehen, spielt das Schuhwerk eine\nzentrale Rolle. Anders gesagt: Ohne Laufschuhe läuft nichts. Warum\ndas so ist und worauf es bei den Modellen ankommt:\n\nReicht für den Start nicht der alte Hallenschuh?\n\nNein. «Ein Hallenschuh ist sicher nicht geeignet, um einen etwa auf\nWaldboden weit zu tragen», erklärt der Orthopäde Patrik Reize. Beim'

In [75]:
df.loc[48620281, 'text_preprocessed'][:500]

'Frische Luft und Bewegung: Diese Kombination reizt viele Menschen am Laufen. Damit die Muskeln und Sehnen eine Joggingrunde gut überstehen, spielt das Schuhwerk eine zentrale Rolle. Anders gesagt: Ohne Laufschuhe läuft nichts. Warum das so ist und worauf es bei den Modellen ankommt: Reicht für den Start nicht der alte Hallenschuh? Nein. «Ein Hallenschuh ist sicher nicht geeignet, um einen etwa auf Waldboden weit zu tragen», erklärt der Orthopäde Patrik Reize. Beim Laufen würden andere Fußteile b'

In [76]:
import string
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/Sanna/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [77]:
# adding number of tokens and mean token length
# (column "wordcount" already exists, slightley differetn outcome but okay)
df['nr_tokens'] = [ len([ t for t in nltk.word_tokenize(text) if t not in string.punctuation])
                    for text in df.text_preprocessed ]

df['mean_token_length'] = [ np.mean([ len(t) for t in nltk.word_tokenize(text) if t not in string.punctuation])
                            for text in df.text_preprocessed ]

/Users/Sanna/miniconda3/envs/work/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/Sanna/miniconda3/envs/work/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [78]:
# add a column: avg time divided by wordcount
df['avgTimeOnPage/wordcount'] = df.avgTimeOnPage/df.wordcount

/Users/Sanna/miniconda3/envs/work/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [79]:
df.head()

,pageviews,entrances,exits,bounces,timeOnPage,conversions,avgTimeOnPage,stickiness,entranceRate,bounceRate,...,text,titelH1,titelH3,wordcount,category,city,text_preprocessed,nr_tokens,mean_token_length,avgTimeOnPage/wordcount
articleId,,,,,,,,,,,,,,,,,,,,,
48620281,21,7,12,7,1012,NaN,112.444444,42.857143,33.333333,33.333333,...,München/Stuttgart (dpa/tmn) - Frische Luft und...,Ohne Treter läuft nichts: Die richtigen Schuhe...,NaN,769,vm,München/Stuttgart,Frische Luft und Bewegung: Diese Kombination r...,796,5.359296,0.146222
48620381,19,6,11,5,1484,NaN,185.500000,42.105263,31.578947,26.315789,...,Berlin/Frankfurt/Main (dpa/tmn) - Der Wecker k...,Ohne Antrieb: Wann ist der Winter-Blues eine D...,NaN,441,vm,Berlin/Frankfurt/Main,"Der Wecker klingelt, aufstehen! Doch gerade im...",452,5.938053,0.420635
48622639,2,2,2,2,0,NaN,0.000000,0.000000,100.000000,100.000000,...,Berlin (dpa/tmn) - Eltern auf der Suche nach e...,Funk statt WLAN: Klassische Babyphones weiter ...,NaN,390,vm,Berlin,Eltern auf der Suche nach einem guten Babyphon...,396,5.848485,0.000000
48623085,32,9,20,9,974,NaN,81.166667,37.500000,28.125000,28.125000,...,Berlin (dpa/tmn) - Spülmaschinentabs sollen kl...,Warentest: Die besten Spülmaschinen-Multitabs ...,NaN,345,vm,Berlin,Spülmaschinentabs sollen kleine Alleskönner se...,367,5.594005,0.235266
48623259,24,2,7,2,3797,NaN,223.352941,70.833333,8.333333,8.333333,...,Berlin (dpa/tmn) - Make-up hat heutzutage eine...,Ist das noch natürlich? - Stiftung Warentest p...,NaN,182,vm,Berlin,Make-up hat heutzutage einen Zweck: Es soll da...,183,5.622951,1.227214


In [80]:
df.city.unique()[:20]

array(['München/Stuttgart', 'Berlin/Frankfurt/Main', 'Berlin', 'München',
       'Frankfurt/Main', 'Goslar', 'Jena', 'Siegburg/Berlin', 'Drebkau',
       'Ulm', 'Göttingen', 'Kassel', 'Stuttgart/Berlin', 'Nürnberg',
       'Offenburg', 'Düsseldorf/Freiburg', 'Essen', 'Berlin/Kerpen',
       'Stuttgart/Klingenthal', 'Bad Honnef'], dtype=object)

In [81]:
df.geoArea.unique()

array(['Deutschland', nan, 'Deutschland__,__Österreich__,__Schweiz',
       'Indien', 'Österreich__,__Schweiz__,__Frankreich', 'Italien',
       'Uganda', 'Deutschland__,__Italien', 'Österreich', 'International'],
      dtype=object)

In [82]:
df.genre.unique()

array(['Fragen & Antworten__,__Fragen und Antworten', 'KORR-Bericht',
       'Zusammenfassung', 'Meldung', 'Themendienst-Gespräch', 'Interview',
       'Fragen & Antworten', 'Hintergrund', 'Umfrage', 'Extra'],
      dtype=object)

In [83]:
df.category.unique() # same as categoryCode, not very informative...

array(['vm'], dtype=object)

In [84]:
df.categoryCode.unique() # same as category, not very informative...

array(['dpacat:vm'], dtype=object)

In [85]:
#df.sort_values(by=['avgTimeOnPage/wordcount'], ascending = False)
df.sort_values(by=['pageviews'], ascending = False)
#df.sort_values(by=['mean_token_length'])

,pageviews,entrances,exits,bounces,timeOnPage,conversions,avgTimeOnPage,stickiness,entranceRate,bounceRate,...,text,titelH1,titelH3,wordcount,category,city,text_preprocessed,nr_tokens,mean_token_length,avgTimeOnPage/wordcount
articleId,,,,,,,,,,,,,,,,,,,,,
49200513,3047,1119,1693,924,217337,NaN,160.514771,44.437151,36.724647,30.324910,...,Köln (dpa/tmn) - Die neuen Fälle von Ansteckun...,Einschränkungen wegen Covid-19: Was Arbeitnehm...,NaN,376,vm,Köln,Die neuen Fälle von Ansteckungen mit dem Coron...,379,5.598945,0.426901
49333693,2487,1877,2032,1699,136927,NaN,300.938462,18.295135,75.472457,68.315239,...,Frankfurt/Main/Düsseldorf (dpa/tmn) - Nachdem ...,Covid-19: Welche Regeln gelten im Betrieb?,NaN,768,vm,Frankfurt/Main/Düsseldorf,Nachdem mehr und mehr Fälle von Covid-19-Infek...,770,6.137662,0.391847
50434977,1956,1865,1848,1779,29972,NaN,277.518519,5.521472,95.347648,90.950920,...,Frankfurt/Main/Kiel (dpa/tmn) - Mitarbeiter in...,Urteil: Rauswurf per Freistellung nicht ohne w...,NaN,181,vm,Frankfurt/Main/Kiel,Mitarbeiter in ungekündigter Anstellung dürfen...,178,6.825843,1.533251
49473155,1887,712,1006,587,169090,NaN,191.929625,46.687864,37.731849,31.107578,...,Düsseldorf (dpa/tmn) - Um die Verbreitung des ...,Veranstaltung wegen Corona abgesagt: Rechte de...,NaN,708,vm,Düsseldorf,Um die Verbreitung des neuen Coronavirus Sars-...,718,5.988858,0.271087
49321583,1709,1226,1333,1085,92555,NaN,246.156915,22.001170,71.737858,63.487420,...,Berlin (dpa/tmn) - Der neuartige Coronavirus S...,Ansteckung und Vorbeugung: Der Alltag mit Covi...,NaN,882,vm,Berlin,Der neuartige Coronavirus Sars-Cov-2 verbreite...,889,5.823397,0.279089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49336939,1,1,1,1,0,NaN,0.000000,0.000000,100.000000,100.000000,...,Wolfsburg (dpa/tmn) - VW erhöht die Spannung u...,Zweites Elektroauto von VW: Der ID4 wird ein SUV,NaN,115,vm,Wolfsburg,VW erhöht die Spannung und bringt sein zweites...,112,5.419643,0.000000
50555957,1,0,0,0,53,NaN,53.000000,100.000000,0.000000,0.000000,...,Hamburg (dpa/tmn) - Bereits ab einem Beschäfti...,Gesundheit: Betriebsärztliche Betreuung ab ein...,NaN,152,vm,Hamburg,Bereits ab einem Beschäftigten muss ein Untern...,155,6.490323,0.348684
49338943,1,0,0,0,38,NaN,38.000000,100.000000,0.000000,0.000000,...,Köln (dpa/tmn) - Die Haut von Kindern ist dünn...,Kind verbrennt sich: Haut mit handwarmem Wasse...,NaN,152,vm,Köln,Die Haut von Kindern ist dünner als die Haut E...,149,5.879195,0.250000


In [86]:
df.describe()

,pageviews,entrances,exits,bounces,timeOnPage,conversions,avgTimeOnPage,stickiness,entranceRate,bounceRate,titelH3,wordcount,nr_tokens,mean_token_length,avgTimeOnPage/wordcount
count,891.000000,891.000000,891.000000,891.000000,891.000000,0.0,891.000000,891.000000,891.000000,891.000000,0.0,891.000000,891.000000,891.000000,891.000000
mean,35.459035,16.163861,20.362514,14.320988,2114.952862,NaN,114.667955,58.255002,28.108763,24.473906,NaN,340.406285,344.732884,5.836816,0.420131
std,189.988507,113.578326,131.153139,102.734228,11761.128249,NaN,125.972488,24.728936,26.065194,24.799523,NaN,211.222006,217.726863,0.385692,0.557882
min,1.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN,74.000000,73.000000,4.665854,0.000000
25%,6.000000,1.000000,2.000000,0.000000,187.000000,NaN,45.600000,43.614130,6.250000,0.000000,NaN,171.000000,169.000000,5.576123,0.166660
50%,10.000000,2.000000,3.000000,2.000000,494.000000,NaN,80.000000,60.000000,21.739130,18.181818,NaN,262.000000,262.000000,5.795322,0.289234
75%,17.000000,5.000000,7.000000,4.000000,1177.500000,NaN,143.440476,75.000000,42.857143,39.144705,NaN,492.500000,504.000000,6.077609,0.474347
max,3047.000000,1877.000000,2032.000000,1779.000000,217337.000000,NaN,1490.000000,100.000000,100.000000,100.000000,NaN,1374.000000,1431.000000,7.260563,6.772727


In [88]:
# saving modified (added columns) DataFrame (df) to encrypted folder
df.to_csv(ROOT / '200707_aachener_zeitung_modified.csv')

## other stuff

In [258]:
df.loc[114].articleId

48953419

In [27]:
df.loc[115].articleId

48953519

In [28]:
df.loc[116].articleId

48955787

In [29]:
df.loc[117].articleId

48956507

In [30]:
df.loc[118].articleId

48960659

In [31]:
df.loc[119].articleId

48960795

## plotting

In [43]:
#plt.scatter(df.mean_token_length, df['avgTimeOnPage/wordcount'])
#plt.show() # Depending on whether you use IPyth